# $a \geq b$ with TTP and hash chains

Alice needs to prove her age to be above a given age without revealing her age to Bob. Alice and Bob trust Trent.

1. Alice tells Trent the age she has to prove. 
2. Trent sends a random seed to her and generates a signed proof statement that she can give to Bob. 
3. Alice hashes her age and passes this to Bob.

In [90]:
from Crypto.Hash import SHA256
from Crypto.Util import number

alice_age = 19 # Do not show to Bob. Send to Trent.
required_age = 18 # Supplied by Bob. Public.

# Trend does the following
seed = bytes(number.Random.get_random_bytes(64))
proof = SHA256.new(data=seed) # Trend should sign this proof with his private key and make it and sig public.
hidden_age = SHA256.new(data=seed) # Trend makes this public

# Trent does this. Note that if Alice is below Bob's required age, Trent only hands over H(seed)
for i in range(1, alice_age - required_age + 1):
    proof = SHA256.new(data=bytes(proof.hexdigest(), encoding='utf-8'))

print(proof.hexdigest())

# Alice gets H(seed) and does this. She sends final value to Bob.
for i in range(1, alice_age + 1):
    hidden_age = SHA256.new(data=bytes(hidden_age.hexdigest(), encoding='utf-8'))

print(hidden_age.hexdigest())

# Bob now does the following
verified_age = proof
for i in range(required_age):
    verified_age = SHA256.new(data=bytes(verified_age.hexdigest(), encoding='utf-8'))

print(verified_age.hexdigest())

f194b8fd48a815e94b61509c30a65e3369a1cebdac024bf9ee01d23ff7bbd3b7
410eb93ea2028fed4097da08c3669de9d91d29747cdbb5b8c34d04cb5e93ed7e
410eb93ea2028fed4097da08c3669de9d91d29747cdbb5b8c34d04cb5e93ed7e


The above works because 

* Alice will get H(seed). She then hashes the seed $m$ times, where $m$ is her age.
* Bob will get H(H...H(seed)) and thus cannot reveal Alice age if Alice is below target. 
* If Alice is below Bob's required age, Bob will get values [H(0), ...,H(n)]
* If Alice has same age as Bob requires or above, then Bob and Alice both hash the seed the same number of times. Bob cannot know what $m-n$ and thus cannot know Alice exact age.

* ZKP requirements:
 * Completeness. Alice will always be able to prove to Bob that she is old enough.
 * Soundness. Alice cannot trick Bob into thinking she is old enough.
 * Zero-knowledge. Bob has no idea about how old Alice is